In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
movie = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movie.merge(credits,on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
movies.duplicated().sum()

np.int64(0)

In [7]:
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [8]:
movies['genres'] = movies['genres'].apply(convert)

In [9]:
movies['keywords'] = movies['keywords'].apply(convert)

In [10]:
def convert_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [11]:
movies['cast'] = movies['cast'].apply(convert)

In [12]:
def fectch_director(text):
  l=[]
  for i in ast.literal_eval(text):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [13]:
movies['crew']=movies['crew'].apply(fectch_director)

In [14]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [15]:
def remove_space(word):
  l=[]
  for i in word:
    l.append(i.replace(" ",""))
  return l

In [16]:
movies['cast']=movies['cast'].apply(remove_space)
movies['crew']=movies['crew'].apply(remove_space)
movies['genres']=movies['genres'].apply(remove_space)
movies['keywords']=movies['keywords'].apply(remove_space)

In [17]:
movies['tags']= movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [18]:
new_df = movies[['movie_id','title','tags']]

In [19]:
new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\91630\AppData\Local\Temp\ipykernel_37672\335926201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x))


In [20]:
new_df['tags']= new_df['tags'].apply(lambda x:x.lower())

C:\Users\91630\AppData\Local\Temp\ipykernel_37672\899821970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']= new_df['tags'].apply(lambda x:x.lower())


In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
def stems(text):
  l=[]
  for i in text.split():
    l.append(ps.stem(i))
  return " ".join(l)

In [23]:
new_df['tags']= new_df['tags'].apply(stems)

C:\Users\91630\AppData\Local\Temp\ipykernel_37672\2470834803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']= new_df['tags'].apply(stems)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [25]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [27]:
def recommend(movie):
  if movie not in new_df['title'].values:
    print(f"Movie '{movie}' not found in the database.")
    return
  index = new_df[new_df['title']==movie].index[0]
  distance =sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])
  for i in distance[1:6]:
    print(new_df.iloc[i[0]].title)

In [28]:
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))